# Python Flask REST API Prototype V 1.0.2  #

*통계 쿼리 -> 시도별 코로나 현황 집계 (누적순위)*

*통계 쿼리 -> 특정 지역 년월 평균 온습도* 

*통계 쿼리 -> 특정 자역 년월 평균 실측파고*

*실시간 현황 -> KMA 코로나팩트*

*통계 쿼리 -> 특정 지역 년월 온습도 중간값*

*통계 쿼리 -> 실종자 통계, 치매, 지역, 연령, 성별 카운트*




In [40]:
'''
CREATE TABLE coronastatus (
  num int(11) NOT NULL AUTO_INCREMENT,
  udate varchar(255) NOT NULL,
  area varchar(100) NOT NULL,
  digit varchar(50) NOT NULL,
  yesterday varchar(50) NOT NULL,
  url varchar(50) NOT NULL,
  rdate datetime DEFAULT NOW(),
  primary key (num)
) ENGINE=InnoDB AUTO_INCREMENT=4 DEFAULT CHARSET=utf8;

CREATE TABLE weather (
  num int(11) NOT NULL AUTO_INCREMENT,
  mdate varchar(50) NOT NULL,
  src varchar(50) NOT NULL,
  area varchar(50) NOT NULL,
  state varchar(50) DEFAULT '-',
  temp varchar(50) DEFAULT '-',
  dis varchar(50) DEFAULT '-',
  rain varchar(50) DEFAULT '-',
  hum varchar(50) DEFAULT '-',
  wdir varchar(255) DEFAULT '-',
  wspeed varchar(255) DEFAULT '-',
  hpa varchar(255) DEFAULT '-',
  posted datetime DEFAULT NOW(),
  primary key (num)
) ENGINE=InnoDB AUTO_INCREMENT=4 DEFAULT CHARSET=utf8;

CREATE TABLE bouy (
  num int(11) NOT NULL AUTO_INCREMENT,
  obs_post_id char(7) NOT NULL,
  obs_post_name varchar(30) NOT NULL,
  obs_lat varchar(10) NOT NULL,
  obs_lon varchar(10) NOT NULL,
  obs_last_req_cnt varchar(11) NOT NULL,
  record_time varchar(19) NOT NULL,
  water_temp varchar(10) NOT NULL,
  salinity varchar(10) NOT NULL,
  air_temp varchar(10) NOT NULL,
  air_pres varchar(10) NOT NULL,     
  wind_dir varchar(10) NOT NULL,
  wind_speed varchar(10) NOT NULL,
  current_dir varchar(10) NOT NULL,
  current_speed varchar(10) NOT NULL,
  wave_height varchar(10) NOT NULL,
  posted datetime DEFAULT NOW(),
  primary key (num)
) ENGINE=InnoDB AUTO_INCREMENT=4 DEFAULT CHARSET=utf8;

CREATE TABLE coronafact (
  num int(11) NOT NULL AUTO_INCREMENT,
  kind varchar(30) NOT NULL,
  udate varchar(255) NOT NULL,
  confirmator varchar(100) NOT NULL,
  dead varchar(50) NOT NULL,
  suspected varchar(50) NOT NULL,
  inspection varchar(50) NOT NULL,
  url varchar(50) NOT NULL,
  rdate datetime DEFAULT NOW(),
  primary key (num)
) ENGINE=InnoDB AUTO_INCREMENT=4 DEFAULT CHARSET=utf8;

CREATE TABLE missing (
  num int(11) NOT NULL AUTO_INCREMENT,
  src varchar(50) NOT NULL,
  seq varchar(50) NOT NULL,
  title varchar(100) NOT NULL,
  thumbnail varchar(255) NOT NULL,
  kind varchar(100) NOT NULL,
  age varchar(100) NOT NULL,
  odate varchar(100) NOT NULL, 
  area varchar(100) NOT NULL,
  wear varchar(100) NOT NULL,  
  posted datetime DEFAULT NOW(),
  primary key (num)
) ENGINE=InnoDB AUTO_INCREMENT=4 DEFAULT CHARSET=utf8;

END'''

"\nCREATE TABLE coronastatus (\n  num int(11) NOT NULL AUTO_INCREMENT,\n  udate varchar(255) NOT NULL,\n  area varchar(100) NOT NULL,\n  digit varchar(50) NOT NULL,\n  yesterday varchar(50) NOT NULL,\n  url varchar(50) NOT NULL,\n  rdate datetime DEFAULT NOW(),\n  primary key (num)\n) ENGINE=InnoDB AUTO_INCREMENT=4 DEFAULT CHARSET=utf8;\n\nCREATE TABLE weather (\n  num int(11) NOT NULL AUTO_INCREMENT,\n  mdate varchar(50) NOT NULL,\n  src varchar(50) NOT NULL,\n  area varchar(50) NOT NULL,\n  state varchar(50) DEFAULT '-',\n  temp varchar(50) DEFAULT '-',\n  dis varchar(50) DEFAULT '-',\n  rain varchar(50) DEFAULT '-',\n  hum varchar(50) DEFAULT '-',\n  wdir varchar(255) DEFAULT '-',\n  wspeed varchar(255) DEFAULT '-',\n  hpa varchar(255) DEFAULT '-',\n  posted datetime DEFAULT NOW(),\n  primary key (num)\n) ENGINE=InnoDB AUTO_INCREMENT=4 DEFAULT CHARSET=utf8;\n\nCREATE TABLE bouy (\n  num int(11) NOT NULL AUTO_INCREMENT,\n  obs_post_id char(7) NOT NULL,\n  obs_post_name varchar(30) NO

In [41]:
!sudo pip3 install pymysql

In [42]:
!sudo pip3 install flask-ngrok

In [43]:
!sudo pip3 install flask_restful

In [44]:
import json
import pymysql
import configparser
from flask import Flask
from decimal import Decimal
from datetime import datetime
from google.colab import drive
from flask_restful import reqparse
from flask_restful import Resource, Api
from flask_ngrok import run_with_ngrok

In [45]:
c=drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [46]:
CNF_INI = 'drive/My Drive/development/config/openapi.ini'
ERR_URI = 'drive/My Drive/development/test/error.log'

In [47]:
config = configparser.ConfigParser()
config.read(CNF_INI)
HOST = config['mysql']['host']
USER = config['mysql']['user']
PASSWORD = config['mysql']['password']
DATABASE = config['mysql']['database']
PORT = config['mysql']['port']
CHARSET = config['mysql']['charset']

In [48]:
def default(obj):
  if isinstance(obj, Decimal):
    return str(obj)
  raise TypeError("Object of type '%s' is not JSON serializable" % type(obj).__name__)

In [49]:
def db_access():
  conn = pymysql.connect(
      host=HOST,
      user=USER,
      password=PASSWORD,
      db=DATABASE,
      charset=CHARSET,
      port=int(PORT),
      cursorclass=pymysql.cursors.DictCursor)
  return conn

In [50]:
def exec_status_corona(conn):
  with conn.cursor() as cursor:
    query = get_status_corona()
    cursor.execute(query)
    data = cursor.fetchall()
    return data  

In [51]:
def exec_status_weather(conn,area,year,month):
  with conn.cursor() as cursor:
    query = get_status_weather(area,year,month)
    cursor.execute(query)
    data = cursor.fetchall()
    return data  

In [52]:
def exec_status_bouy(conn,area,year,month):
  with conn.cursor() as cursor:
    query = get_status_bouy(area,year,month)
    cursor.execute(query)
    data = cursor.fetchall()
    return data  

In [53]:
def exec_lately_corona(conn):
  with conn.cursor() as cursor:
    query = get_lately_corona()
    cursor.execute(query)
    data = cursor.fetchall()
    return data  

In [54]:
def exec_median_weather(conn,area,year,month,branch):
  with conn.cursor() as cursor:
    query = get_median_weather(area,year,month,branch)
    cursor.execute(query)
    data = cursor.fetchall()
    return data  

In [55]:
def exec_count_missing(conn,kind,area):
  with conn.cursor() as cursor:
    query = get_count_missing(kind,area)
    cursor.execute(query)
    data = cursor.fetchall()
    return data  

In [56]:
def get_status_corona():
  query = 'select * from \
  (select area, sum(convert(replace(digit, \',\',\'\'), unsigned)) as digit from coronastatus group by area order by digit desc) as a \
  join \
  (select udate as start from coronastatus where num in(select min(num) from coronastatus)) as b \
  join \
  (select udate as end from coronastatus where num in(select max(num) from coronastatus)) as c'
  print(query)
  return query 

In [57]:
def get_status_weather(area,year,month):
  query = 'select area, substr(mdate,1,4) as year, substr(mdate,6,2) as month, \
  round(avg(temp),2) as temp, round(avg(hum),2) as hum from weather \
  where area=\'{}\' and substr(mdate,1,4) = \'{}\' and substr(mdate,6,2) = \'{}\' '.format(area,year,month)
  print(query)
  return query 

In [58]:
def get_status_bouy(area,year,month):
  query = 'select \
  obs_post_name as area, substr(record_time,1,4) as year, substr(record_time,6,2) as month, \
  round(avg(replace(water_temp,\'None\',0)),1) as water_temp, \
  round(avg(replace(salinity,\'None\',0)),1) as salinity, \
  round(avg(replace(air_temp,\'None\',0)),1) as air_temp, \
  round(avg(replace(air_pres,\'None\',0)),1) as air_pres, \
  round(avg(replace(wind_dir,\'None\',0)),1) as wind_dir, \
  round(avg(replace(wind_speed,\'None\',0)),1) as wind_speed, \
  round(avg(replace(current_dir,\'None\',0)),1) as current_dir, \
  round(avg(replace(current_speed,\'None\',0)),1) as current_speed, \
  round(avg(replace(wave_height,\'None\',0)),1) as wave_height from bouy \
  where obs_post_name like \'%{}%\' and substr(record_time,1,4) = \'{}\' \
  and substr(record_time,6,2) = \'{}\''.format(area,year,month)
  print(query)
  return query 

In [59]:
def get_lately_corona():
  query = 'select udate, confirmator, dead, suspected, inspection from coronafact order by num desc limit 1'
  print(query)
  return query 

In [60]:
def get_median_weather(area,year,month,branch):
  query = 'select avg(dd.{}) as median_{} \
  from ( \
  select d.{}, @rownum:=@rownum+1 as `row_number`, @total_rows:=@rownum \
  from weather d, (select @rownum:=0) r \
  where d.{} is not null \
  and area=\'{}\' and substr(mdate,1,4) = \'{}\' and substr(mdate,6,2) = \'{}\' \
  order by d.{} \
  ) as dd \
  where dd.row_number IN ( floor((@total_rows+1)/2), floor((@total_rows+2)/2) )'.format(branch,branch,branch,branch,area,year,month,branch)
  print(query)
  return query 

In [61]:
def get_count_missing(kind,area):
  query = 'select substr(age,2,1) as age , substr(title,-2,2) as sex, count(*) as cnt \
  from missing where kind=\'{}\' and area like \'%{}%\' \
  group by substr(age,2,1), substr(title,-2,2) order by age desc'.format(kind,area)
  print(query)
  return query 

In [62]:
class confirm_api(Resource):
  def get(self):
    return {'status':'200'}

In [63]:
class status_corona(Resource):
  def get(self):
    try:
      
      conn = db_access()

      data = json.dumps(exec_status_corona(conn), default=default, ensure_ascii=False)

      print(data)
      
      if data:
        return {'staus': '301', 'message': 'I have a data', 'data': data}  
      else:
        return {'staus': '300', 'message': 'There are no data.'}    

    except Exception as e:
      with open(ERR_URI,'a') as file:
        file.write('{} You got an error: {}\n'.format(datetime.today().strftime('%Y-%m-%d %H:%M:%S'),str(e)))   

In [64]:
class status_weather(Resource):
  def get(self):
    try:
      parser = reqparse.RequestParser()
      parser.add_argument('area', default='부산', type=str)
      parser.add_argument('year', default='2020', type=str)
      parser.add_argument('month', default='08', type=str)
      args = parser.parse_args()
      
      area = args['area']
      year = args['year']
      month = args['month']     

      conn = db_access()

      # data = json.dumps(exec_status_corona(conn), default=default, ensure_ascii=False)
      data = exec_status_weather(conn,area,year,month)

      print(data)
      
      if data:
        return {'staus': '301', 'message': 'I have a data', 'data': data}  
      else:
        return {'staus': '300', 'message': 'There are no data.'}    

    except Exception as e:
      with open(ERR_URI,'a') as file:
        file.write('{} You got an error: {}\n'.format(datetime.today().strftime('%Y-%m-%d %H:%M:%S'),str(e)))   

In [65]:
class status_bouy(Resource):
  def get(self):
    try:
      parser = reqparse.RequestParser()
      parser.add_argument('area', default='해운대', type=str)
      parser.add_argument('year', default='2020', type=str)
      parser.add_argument('month', default='08', type=str)
      args = parser.parse_args()
      
      area = args['area']
      year = args['year']
      month = args['month']     

      conn = db_access()

      # data = json.dumps(exec_status_corona(conn), default=default, ensure_ascii=False)
      data = exec_status_bouy(conn,area,year,month)

      print(data)
      
      if data:
        return {'staus': '301', 'message': 'I have a data', 'data': data}  
      else:
        return {'staus': '300', 'message': 'There are no data.'}    

    except Exception as e:
      with open(ERR_URI,'a') as file:
        file.write('{} You got an error: {}\n'.format(datetime.today().strftime('%Y-%m-%d %H:%M:%S'),str(e)))   

In [66]:
class lately_corona(Resource):
  def get(self):
    try:
      
      conn = db_access()

      data = exec_lately_corona(conn)

      print(data)
      
      if data:
        return {'staus': '401', 'message': 'I have a data', 'data': data}  
      else:
        return {'staus': '400', 'message': 'There are no data.'}    

    except Exception as e:
      with open(ERR_URI,'a') as file:
        file.write('{} You got an error: {}\n'.format(datetime.today().strftime('%Y-%m-%d %H:%M:%S'),str(e)))   

In [67]:
class median_weather(Resource):
  def get(self):
    try:
      parser = reqparse.RequestParser()
      parser.add_argument('area', default='부산', type=str)
      parser.add_argument('year', default='2020', type=str)
      parser.add_argument('month', default='08', type=str)
      parser.add_argument('branch', default='temp', type=str)
      args = parser.parse_args()
      
      area = args['area']
      year = args['year']
      month = args['month']
      branch = args['branch']    

      conn = db_access()

      # data = json.dumps(exec_status_corona(conn), default=default, ensure_ascii=False)
      data = exec_median_weather(conn,area,year,month,branch)

      print(data)
      
      if data:
        return {'staus': '501', 'message': 'I have a data', 'data': data}  
      else:
        return {'staus': '500', 'message': 'There are no data.'}    

    except Exception as e:
      with open(ERR_URI,'a') as file:
        file.write('{} You got an error: {}\n'.format(datetime.today().strftime('%Y-%m-%d %H:%M:%S'),str(e))) 

In [70]:
class count_missing(Resource):
  def get(self):
    try:
      parser = reqparse.RequestParser()
      parser.add_argument('kind', default='치매', type=str)
      parser.add_argument('area', default='부산', type=str)
      args = parser.parse_args()
      
      kind = args['kind']
      area = args['area']  

      conn = db_access()

      data = exec_count_missing(conn,kind,area)

      print(data)
      
      if data:
        return {'staus': '601', 'message': 'I have a data', 'data': data}  
      else:
        return {'staus': '600', 'message': 'There are no data.'}    

    except Exception as e:
      with open(ERR_URI,'a') as file:
        file.write('{} You got an error: {}\n'.format(datetime.today().strftime('%Y-%m-%d %H:%M:%S'),str(e)))   

## Howto ##

통계 쿼리 : 시도별 코로나 현황 집계 (누적순위) /corona/status

통계 쿼리 : 특정 지역, 년, 월 평균 온습도 /weather/status?area=부산&year=2020&month=07

통계 쿼리 : 특정 지역, 년, 월 평균 실측파고 /bouy/status?area=해운대&year=2020&month=07

실시간 현황 : KMA 코로나팩트 /corona/lately

통계 쿼리 : 특정 지역, 년, 월 온습도 중간값 /weather/median?area=부산&year=2020&month=07&branch=temp

통계 쿼리 : 특정 지역, 연령별, 성별, 실종자 카운트 /missing/count?kind=치매&area=부산

In [71]:
app = Flask(__name__)
api = Api(app)
run_with_ngrok(app)
api.add_resource(confirm_api,'/')
api.add_resource(status_corona,'/corona/status/')
api.add_resource(status_weather,'/weather/status/')
api.add_resource(status_bouy,'/bouy/status/')
api.add_resource(lately_corona,'/corona/lately/')
api.add_resource(median_weather,'/weather/median/')
api.add_resource(count_missing,'/missing/count/')
app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://1bb6b71e3896.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [21/Aug/2020 01:07:09] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [21/Aug/2020 01:07:10] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [21/Aug/2020 01:07:23] "GET /missing/count?kind=아동&area=부산 HTTP/1.1" 308 -
127.0.0.1 - - [21/Aug/2020 01:07:24] "GET /missing/count/?kind=아동&area=부산 HTTP/1.1" 200 -


select substr(age,2,1) as age , substr(title,-2,2) as sex, count(*) as cnt   from missing where kind='아동' and area like '%부산%'   group by substr(age,2,1), substr(title,-2,2) order by age desc
[{'age': '9', 'sex': '남자', 'cnt': 2}, {'age': '8', 'sex': '남자', 'cnt': 1}, {'age': '7', 'sex': '남자', 'cnt': 1}, {'age': '4', 'sex': '여자', 'cnt': 1}, {'age': '1', 'sex': '남자', 'cnt': 10}, {'age': '1', 'sex': '여자', 'cnt': 25}]
